In [1]:
kernel.silent(true) // supress all non-explicit output
import ammonite.ops._
val jarString = "../target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar"
val path = ammonite.ops.pwd / ammonite.ops.RelPath(jarString)
interp.load.cp(path)


import ammonite.ops._

jarString: String = "../target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar"
path: Path = /Users/michaelmccoy/Projects/euclid-scala/target/scala-3.0.2/euclid-scala-assembly-0.1.0-SNAPSHOT.jar

In [2]:
def makeExponents(order: Int, l: List[Int]): List[(Int, Int)] = {
    l.zipWithIndex.map(t => (t._2, t._1)) :::((order, 1) :: Nil)
}

In [3]:
import com.mbmccoy.euclid.polynomial._
import scala.annotation.tailrec

val prime = Prime.next(65537)
val order = 1
val cf = FiniteField(prime, order)
println(cf.conwayPolynomial)
println(cf.primeField)


1 + X
com.mbmccoy.euclid.polynomial.PrimeField(65537)


In [4]:
def getOrder(x: cf.Element): Int = {
    var y = x
    var order = 1
    while (y != cf.one) {
        y *= x
        order += 1
    }
    order
}
    
// Print some orders here
var x = cf.one
for (_ <- 0 until 10) {
    val order = getOrder(x)
    println(f"x=${x}, order=${order}")
    x = x + cf.one
}

x=1, order=1
x=2, order=32
x=3, order=65536
x=4, order=16
x=5, order=65536
x=6, order=65536
x=7, order=65536
x=8, order=32
x=9, order=32768
x=10, order=65536


In [5]:
val omega_hat = cf((0, 16384) :: Nil) // 128, order = 32
val omega = omega_hat.pow(4) // order = 8

def printOrder(x: cf.Element) = println(f"${x}: order=${getOrder(x)}")

printOrder(omega_hat)
printOrder(omega)


16384: order=16
65281: order=4


In [6]:

val M = cf((0, 44734)::Nil)
val K1 = cf((0, 34369)::Nil)
val K2 = cf((0, 51062)::Nil)
val MInverse = M.reciprocal

// TODO: verify that this is close enough to a width-1 Rescue hash.
def rescueIteration(x: cf.Element): cf.Element = {
    val y = M * x.pow(3) + K1
    val cubeRoot = y.reciprocal.pow((cf.order.toInt - 2)/3)
    cubeRoot * MInverse + K2
}

def rescueHash(x: cf.Element, rounds: Int): cf.Element = {
    if (rounds <= 0) { 
        x
    } else {
        rescueHash(rescueIteration(x), rounds - 1)
    }
}


In [7]:
val fortyTwo = cf((0, 42)::Nil)

@tailrec
def winterfellReversed(x: cf.Element, rounds: Int, trace: List[cf.Element] = Nil): List[cf.Element] = {
    if (rounds <= 0) trace
    else {
        winterfellReversed(x*x*x + fortyTwo, rounds - 1, x :: trace)
    }
}

In [8]:
val secret = cf((0, 54234)::Nil)
val executionTrace = winterfellReversed(secret, 4).reverse
println(executionTrace)


List(54234, 2820, 55234, 23285)


In [9]:

def getSubgroup(x: cf.Element) = {
    lazy val subgroup: LazyList[cf.Element] = x #:: subgroup.map{y => x*y}
    cf.one :: subgroup.takeWhile(_ != cf.one).toList
}
val omegas = getSubgroup(omega)

println(omegas)

List(1, 65281, 65536, 256)


In [ ]:
import com.mbmccoy.euclid.polynomial.FieldOps.lagrangeInterpolation
println(omegas.zip(executionTrace))
println(lagrangeInterpolation(omegas.zip(executionTrace)))

List((1,54234), (65281,2820), (65536,55234), (256,23285))
